# Декораторы и модуль functools
### Декораторы
* Декоратор - функция, которая принимает другую функцию и что-то возвращает

In [6]:
def trace(*args):
    pass

@trace
def foo(x):
    return 42

#аналогично

def foo(x):
    return 42

foo = trace(foo)
print(type(foo))

<class 'NoneType'>


Декоратор **trace** выводит на экран сообщение с информацией о вызове декорируемой функции 

In [7]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner

@trace
def identity(x):
    "I do smth useful"
    return x

identity(42)

identity (42,) {}


42

### Декораторы и help
* проблема с **help** и атрибутами декорируемой функции
* возможность глобально отключать **trace** 
* явное указание файла при использовании **trace**

In [8]:
help(identity)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [31]:
import sys

@trace(sys.stderr)
def identity(x):
    return x

**__module__** содержит имя модуля, в котором функция была определена

In [11]:
identity.__module__

'__main__'

In [13]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    inner.__module__ = func.__module__
    inner.__name__ = func.__name__
    inner.__doc__ = func.__doc__
    return inner

@trace
def identity(x):
    "I do smth useful"
    return x

identity.__name__, identity.__doc__

('identity', 'I do smth useful')

В модуле functools из стандартной библиотеки Python есть функция, реализующая логику копирования внутренних атрибутов

In [14]:
import functools
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    functools.update_wrapper(inner, func) 
    return inner

То же самое можно сделать с помощью декоратора wraps

In [15]:
def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs) 
    return inner

### Управление поведением trace
* Заведём глобальную переменную trace_enabled и с её помощью будем отключать и включать trace.
* Если trace выключен, то результатом применения декоратора является сама функция func — никаких дополнительных действий в момент её исполнения производиться не будет.

In [24]:
trace_enabled = False
def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner if trace_enabled else func

@trace
def identity(x):
    return x

print(identity(5))
trace_enabled = True
f = trace(identity)
print(f(5))

5
identity (5,) {}
5


### Декораторы с аргументами

In [30]:
@trace(sys.stderr)
def identity(x):
    return x

# аналогично

def identity(x):
    return x

deco = trace(sys.stderr)
identity = deco(identity)

In [25]:
def trace(handle):
    def decorator(func): 
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs,
                  file=handle)
            return func(*args, **kwargs) 
        return inner
    return decorator

Можно обобщить логику декоратора с аргументами в виде декоратора with_arguments

Что происходит:
1. with_arguments принимает декоратор deco,
2. заворачивает его во wrapper, так как deco — декоратор с аргументами, а затем в decorator.
3. decorator конструирует новый декоратор с помощью deco и копирует в него внутренние атрибуты функции func.

In [26]:
def with_arguments(deco):
    @functools.wraps(deco)
    def wrapper(*dargs, **dkwargs):
        def decorator(func):
            result = deco(func, *dargs, **dkwargs) 
            functools.update_wrapper(result, func) 
            return result
        return decorator 
    return wrapper

In [29]:
@with_arguments
def trace(func, handle):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs, file=handle)
        return func(*args, **kwargs)
    return inner

@trace(sys.stderr)
def identity(x):
    return x


identity(42)

identity (42,) {}


42

In [33]:
def trace(func=None, *, handle=sys.stdout):
    # со скобками
    if func is None:
        return lambda func: trace(func, handle=handle)
    
    # без скобок
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs) 
    return inner

Декораторы бывают:
* без аргументов @trace
* с аргументами @trace(sys.stderr) 
* с опциональными аргументами.

### Практика декораторов

In [35]:
import time

def timethis(func=None, *, n_iter = 100):
    if func is None:
        return lambda func: timethis(func, n_iter = n_iter)
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end = "...")
        acc = float('inf')
        for i in range(n_iter):
            tick = time.perf_counter()
            result = func(*args, **kwargs)
            acc = min(acc, time.perf_counter() - tick)
        print(acc)
        return result
    return inner

result = timethis(sum)(range(10 ** 6))

sum...0.023777214984875172


In [38]:
def profiled(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    inner.ncalls = 0
    return inner
    
@profiled
def identity(x):
    return x

print(identity(42))
print(identity.ncalls)

42
1


In [40]:
def once(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        if not inner.called:
            func(*args, **kwargs)
            inner.called = True
            
    inner.called = False
    return inner

@once
def initialize_settings():
    print('Setting init')
    
print(initialize_settings())
print(initialize_settings())

Setting init
None
None


Мемоизация — сохранение результатов выполнения функции для предотвращения избыточных вычислений.

Напишем декоратор для автоматической мемоизации “любой” функции.

In [50]:
def memoized(func):
    cache = {}
    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args, kwargs 
        if key not in cache:
            cache[key] = func(*args, **kwargs) 
        return cache[key]
    return inner


@memoized
def ackermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackermann(m - 1, 1)
    else:
        return ackermann(m - 1, ackermann(m, n - 1))
    
    
ackermann(3, 4)    

TypeError: unhashable type: 'dict'

Частное решение проблемы:

* Нет универсального и быстрого решения. Можно сериализовывать аргументы в строку, например, через str или, что более осмысленно, через pickle.

In [51]:
def memoized(func):
    cache = {}
    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args + tuple(sorted(kwargs.items())) 
        if key not in cache:
            cache[key] = func(*args, **kwargs) 
        return cache[key]
    return inner

@memoized
def ackermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackermann(m - 1, 1)
    else:
        return ackermann(m - 1, ackermann(m, n - 1))
    

ackermann(3, 4) 

125

In [55]:
import warnings

def deprecated(func):
    code = func.__code__
    warnings.warn_explicit(
        func.__name__ + " is deprecated.", 
        category=DeprecationWarning, 
        filename=code.co_filename, 
        lineno=code.co_firstlineno + 1)
    return func

@deprecated
def identity(x):
    return x

identity(2)

2

Контрактное программирование — способ проектирования программ, основывающийся на формальном описании интерфейсов в терминах предусловий, постусловий и инвариантов.

* В Python контрактное программирование можно реализовать в виде библиотеки декораторов:

In [61]:
import math

def pre(cond, message):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            assert cond(*args, **kwargs), message
            return func(*args, **kwargs)
        return inner
    return wrapper

@pre(lambda x: x >= 0, "negative argument")
def checked_log(x):
    return math.log(x)

checked_log(1)
checked_log(-1)

AssertionError: negative argument

In [63]:
def post(cond, message):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            result = func(*args, **kwargs)
            assert cond(result), message
            return result
        return inner
    return wrapper

@post(lambda x: not math.isnan(x), 'not a number')
def smth_useful():
    return float('nan')

smth_useful()

AssertionError: not a number

Cинтаксис Python разрешает одновременное применение нескольких декораторов.
*  Порядок декораторов имеет значение:

In [66]:
def square(func):
    return lambda x : func(x * x)

def addsome(func):
    return lambda x : func(x + 42)

@square
@addsome
def identity(x):
    return x

print(identity(2))

@addsome
@square
def identity(x):
    return x

print(identity(2))

46
1936


## functools

Родственник уже рассмотренного memoized, сохраняющий фиксированное количество последних вызовов - это декоратор lru_cache

Можно не ограничивать количество сохраняемых вызовов, тогда мы получим в точности поведение memoized:

In [68]:
@functools.lru_cache(maxsize=64)
def ackermann(m, n):
    pass

ackermann.cache_info()

@functools.lru_cache(maxsize=None)
def ackermann(m, n):
    pass

С помощью partial можно зафиксировать часть позиционных и ключевых аргументов в функции.

In [69]:
f = functools.partial(sorted, key=lambda p: p[1])
f([("a", 4), ("b", 2)])

[('b', 2), ('a', 4)]

In [71]:
g = functools.partial(sorted, [2, 3, 1, 4])
g()

[1, 2, 3, 4]

Функция **len** называется **обобщённой**, так как её реализация может быть специализирована для конкретного типа. **str, hash, sum** - также обощенные функции


* В качестве примера реализуем функцию pack, которая сериализует объект в компактное строковое представление

In [74]:
@functools.singledispatch
def pack(obj):
    type_name = type(obj).__name__
    assert False, "Unsupported type: " + type_name
    
    
@pack.register(int)
def _(obj):
    return b"I" + hex(obj).encode("ascii")

@pack.register(list)
def _(obj):
    return b"L" + b",".join(map(pack, obj))

print(pack([1, 2, 3]))
print(pack(42.))

b'LI0x1,I0x2,I0x3'


AssertionError: Unsupported type: float

* Функция reduce обобщает логику функции sum на произвольную бинарную операцию
* Функция reduce принимает три аргумента: бинарную функцию, последовательность и опциональное начальное значение.

In [75]:
functools.reduce(lambda acc, x: acc * x, [1, 2, 3, 4])

24

Несмотря на свою популярность в функциональных языках, в Python довольно сложно придумать полезный пример использования reduce.
* Резюме про reduce:
    * работает с любым объектом,поддерживающим протокол итератора;
    * работает слева направо;
    * использует первый элемент последовательности,если начальное значение не указано явно.